<div style='color: #690027;' markdown="1">
    <h1>Van blad naar label: stomatadetectie</h1> 
</div>

In deze notebook kan je je eigen microfoto's uploaden en testen in het diep neuraal netwerk dat besproken wordt in de paper [*From leaf to label: a robust automated workflow for stomata detection* van *Sofie Meeus, Jan Van den Bulcke and Francis wyffels*](https://doi.org/10.1002/ece3.6571).
<img src="https://drive.google.com/uc?export=view&id=1g1wIt37A07yDi7w9uCza3eMt1oEPLJ5v" alt="Overview" style="width:600px;"/>
Zoals hierboven geïllustreerd glijdt een venster over jouw microfoto (A). Dit *sliding window* verdeelt jouw foto dus in kleine overlappende vlakken of patches (B). Er is een diep neuraal netwerk (VGG19) getraind om deze patches te labelen (C). Positief gelabelde patches van een microfoto  worden geclusterd (D),  wat uitmondt in de detectie (E). Deze detectie is afhankelijk van de drempelwaarde, de *threshold*, die je koos.

Om te starten voer je de volgende code-cel uit door te klikken op de "run"-knop of door shift-enter te gebruiken.

Voor je start eerst nog een tip : als je deze notebook op Colab uitvoert, dan wil je misschien gebruikmaken van de GPU-versnelling. Hiervoor kies je in het menu: *Edit* > *Notebook Settings* > *Hardware accelerator GPU* > *Save*.

### Nodige modules importeren

Je start met het inladen van enkele Python-modules:

- [PIL](https://pillow.readthedocs.io/en/stable/): een handige Python-module om te werken met beelden;
- [NumPy](https://numpy.org): de badismodule om wetenschappelijke bewerkingen in Python uit te voeren;
- [sklearn](https://scikit-learn.org/stable/): de scikit-learn module voor machinaal leren, in het bijzonder voor de functionaliteit van het clusteren;
- [os](https://docs.python.org/3/library/os.html): een Python-module voor functionaliteiten die afhankelijk zijn van het besturingssysteem, bv. lezen, schrijven en bestanden oplijsten;
- [Matplotlib](https://matplotlib.org): een Python-module om grafieken te maken.

Een diep neuraal netwerk bestaat uit meerdere lagen die aaneengeschakeld zijn. De Python-module Keras voorziet bouwblokken om een neuraal netwerk op te bouwen. In de achterliggende code zijn de nodige functionaliteiten vervat. Voor het rekenen met tensoren en andere rekenkundige bewerkingen doet Keras zelf een berope op het platform TensorFlow.

In [ ]:
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
import os
import matplotlib.pyplot as plt

Om te specifiëren welke NVidia GPU je zult gebruiken, is het nodig van de volgende instructies uit te voeren. Dit veronderstelt dat bij Keras en TensorFlow het gebruik van een GPU ingeschakeld is. Voor meer informatie zie de [TensorFlow](https://www.tensorflow.org/install/gpu) documentatie.

In [ ]:
# Als er een NVidia GPU is die je kunt gebruiken, voer dit dan uit
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Vervolgens laden we de verschillende [Keras](https://keras.io/getting_started/intro_to_keras_for_researchers/)-modules in.

In [ ]:
from keras import backend as K
from keras.layers import Input, Convolution2D, Conv2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model, load_model
from keras.engine.topology import get_source_inputs
from keras.utils import get_file
from keras.utils import layer_utils
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

### Dataset

Om een *deep learning*-model te kunnen trainen, heb je data nodig. Zoals eerder vermeld, zal het deep learning-model stomata detecteren op vierkante patches van 120 op 120 pixels. Dat komt omdat het getraind wordt met zo'n patches. Om een robuust model te bekomen, moeten zowel positieve als negatieve voorbeelden aan het systeem worden gepresenteerd.

De data wordt opgesplitst in drie delen:
- De trainingset, dit zijn de data die gebruikt wordt om de gewichten, de *weights*, van het (diep) neuraal netwerk aan te passen;
- De validatieset, dit zijn de data waarmee wordt gekeken hoe goed het leerproces vordert en om de hyperparameters van het model fijner af te stellen;
- De testset, dit zijn de data die je na de training aan het systeem geeft om het ontwikkelde model te testen.

Deze notebook bevat de training en de validering van het *deep learning*-systeem voor stomatadetectie, en een kleine dataset die beperkt is tot *Carapa procera* en geschikt is voor didactische doeleinden. Hierdoor zijn ook de computationele noden binnen de perken gehouden (een volledige training met meerdere plantensoorten (zie de paper) vergt meer tijd en geduld).

Na de training zou het model in principe moeten getest worden op de testset. In deze notebook wordt die test beperkt tot één afbeelding. Dit omwille van de tijd en omdat dit volstaat voor het doeleinde van deze notebook: demonstreren hoe een convolutioneel neuraal netwerk voor stomatadetectie opgebouwd, getraind, gevalideerd en tot slot ingezet wordt.

Download eerst de dataset en unzip hem:

In [ ]:
!wget https://zenodo.org/record/3902280/files/data.zip
!unzip "data.zip"

In [ ]:
train_dir = "./data/training/"
val_dir = "./data/validation/"

De trainings- en validatiedata bevatten patches van 120 op 120 pixels. Een positief gelabelde patch vertoont een stoma:

<img src="https://drive.google.com/uc?export=view&id=11oZG14b8ZnbzooeeQpYaQeUe8wSbErmV"  width="120" />
    
Een negatief gelabelde patch van *Carapa procera* heeft geen stoma (tenzij misschien een deel ervan):

<img src="https://drive.google.com/uc?export=view&id=1vwcYiZDJffjQ6gXCpThwvbcCoPhgseBS"  width="120" />

Om zulke patches te bekomen, moet je beschikken over geannoteerde microfoto's (microfoto's waarvan je de coördinaat kent van de middens van de aanwezige stomata). De patches kunnen dan, gebaseerd op deze coördinaten, uitgesneden worden door middel van de [*crop*-functie](https://pillow.readthedocs.io/en/stable/reference/Image.html) van PIL of nog eenvoudiger door gebruik te maken van [*matrix slicing*](https://numpy.org/doc/1.18/reference/arrays.indexing.html) in NumPy.


Het aantal elementen in de dataset wordt vergroot door middel van *data augmentation*. De preprocessor [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class) definieert de *data augmentation* die toegepast zal worden op de dataset. Hier bestaat die uit willekeurige rotaties, en horizontale en verticale *flips* van de patches.

Behalve het bepalen van de *data augmentation* die zal worden toegepast, gebruik je de ImageDataGenerator ook om enkele zaken vast te leggen: de afmetingen van de patches (120 x 120 pixels), de kleurenmodus (grijswaarden of rgb), de grootte van de batch (dit is het aantal samples dat gebruikt wordt in een epoch van de training, dus in elke trainingiteratie), het classificatietype van de te volbrengen taak (hier binaire classificatie: een patch verdient ofwel een positief ofwel een negatief label), of de data geshuffeld moet worden of niet, en de *seed*, het startpunt, van de willekeurige getalgenerator. Tot slot voorzie je een pad naar de map met de training data.

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=180, horizontal_flip=True, vertical_flip=True, rescale=1/255.)

In [ ]:
batch_size = 128

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./data/training/",
    target_size=(120, 120),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=53
)

Ook voor de validatie stel je een ImageDataGenerator in. Deze definieer je met dezelfde eigenschappen als diegene voor de training maar zonder *data augmentation*.

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255.)

validation_generator = validation_datagen.flow_from_directory(
        r"./data/validation/",
        target_size=(120, 120),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="binary")

### Netwerkarchitectuur met nodige parameters

Je vertrekt van het convolutioneel neuraal netwerk van het [VGG19-model](https://arxiv.org/abs/1409.1556) waar je twee *dense layers* aan toevoegt. De convolutionele neurale lagen zijn voorgetraind op [ImageNet](https://ieeexplore.ieee.org/abstract/document/5206848). Bijgevolg moeten enkel de *dense layers* nog getraind worden. De voorgetrainde gewichten van het convnet download je van Keras via het sleutelwoord "imagenet".

In [ ]:
number_dense_neurons = 2048

In [ ]:
# netwerkarchitectuur
from keras.applications import VGG19
# Je start van een VGG19 als basis (convolutionele neurale lagen) met gewichten voorgetraind op ImageNet
vgg19_base = VGG19(weights="imagenet", include_top=False, input_shape=(120,120,3))
x = vgg19_base.output
x = Flatten()(x)
x = Dense(2*number_dense_neurons, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(number_dense_neurons, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1,activation="sigmoid")(x)

model = Model(inputs=vgg19_base.input, outputs=x)


for layer in vgg19_base.layers:
    layer.trainable = False

model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Train het model

De parameters worden geoptimaliseerd door beroep te doen op de optimalisatiefunctie [Adam](https://arxiv.org/pdf/1412.6980.pdf); hiervoor werd de *learning rate* gefinetuned en uiteindelijk op 0.000005 afgesteld. Tot slot leg je de *loss* en de *metrics* voor training en validatie vast.

In [ ]:
learning_rate = 0.000005
# Initialiseer Stochastic Gradient Descent met momentum, learning rate om te finetunen
opt = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)
# Bepaal de loss en metrics voor training en validatie
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy"])

Training gebeurt door middel van de functie *fit()* gedurende 50 epochs. Merk op dat de architectuur op zo'n manier werd geconfigureerd dat enkel de gewichten van de dense layers aangepast worden. 

In [ ]:
epochs = 50
history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator)

Nu ben je toe aan de volgende stap: het netwerk is getraind en het kan gebruikt worden. Om het AI-systeem te kunnen gebruiken, moet je de parameters van het model opslaan. Dit kan je doen door de instructie *model.save(path)* met *path* het pad naar het bestand waarin je de parameters wilt bewaren. Bovendien geeft de functie *fit()* een *history* object terug. Dit object omvat de vooruitgang van de training en van de validatie over de verschillende epochs. Bijgevolg is dit nuttig om het trainingsproces in het oog te houden, bijvoorbeeld om de resultaten van verschillende instellingen van de hyperparameters, zoals de *learning rate*, het aantal *epochs* en de grootte van de *batches*, te vergelijken.

In [ ]:
# Bewaar het Carapa procera deep learning model
model.save("my_carapa_procera_model")

# Plot de training en validation losses over de verschillende epochs
plt.figure()
plt.title("Training en validation loss van het Carapa procera model over 50 epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(history.history["loss"], label="Training loss")
plt.plot(history.history["val_loss"], label="Validation loss")
plt.legend()
plt.show()

### Laad een deep learning model in

Nu heb je een eerste deep learning model voor stomatadetectie bij de *Carapa procera* getraind. Dit model is opgeslagen als het object *model*. Als je van een gesaved deep learning model wilt vertrekken, dan haal je het op vanuit de file door de instructie *model = load_model(path_to_model)* uit te voeren.

### Beeld- en detectieparameters

Het model kan enkel stomata detecteren op afbeeldingen van 120 op 120 pixels, daarom moet een aangeboden afbeelding ook eerst verdeeld worden in patches. Het model maakt daarvoor gebruik van een werkwijze met een *sliding window*. Hoewel deze methode niet de meest (computationeel) efficiënte is, is ze zeer gemakkelijk te begrijpen. Het venster is 120 op 120 pixels groot en verschuift telkens met een stap van 10 pixels. Je start door je afbeelding in te laden:

In [ ]:
demo_image = "./data/Carapa_procero_demo.jpg" # Je kan een andere Carapa procero microfoto gebruiken

In [ ]:
image = Image.open(demo_image)
fig, ax = plt.subplots(figsize=(20, 10))
image = np.array(image) # Omzetten naar een Numpy array
ax.imshow(image)

In [ ]:
shift = 10
patch_size = 120

Ook het aantal slides dat uitgevoerd wordt, maakt deel uit van de detectieparameters:

In [ ]:
no_x_shifts = (np.shape(image)[0] - patch_size) // shift
no_y_shifts = (np.shape(image)[1] - patch_size) // shift
print("Er gebeuren " + str(no_x_shifts*no_y_shifts) + " slides. Bijgevolg wordt het deep learning model op " + str(no_x_shifts*no_y_shifts) + " patches toegepast.")

### Classificatie met een deep learning model

Nu alle vensters geïdentificeerd zijn, kan het deep learning model in actie treden. Je bewerkstelligt dit door de functie *predict()* te aanroepen. Weliswaar moet je de gebruikte afbeelding eerst converteren (omzetten naar het verwachte formaat) en normaliseren (elementen krijgen waarden van 0 t.e.m. 1). De output van het deep learning model is een getal tussen 0 en 1, het geeft weer hoe zeker het model is dat de afbeelding een stoma vertoont. Daarom moet je ook een drempelwaarde, *threshold*, vastleggen vanaf wanneer de output als een positieve classificatie wordt geaccepteerd. Hoe hoger deze threshold, hoe strenger het systeem zal handelen bij het detecteren van de stomata. Als de threshold echter te hoog is, zal het systeem niet in staat zijn om ook maar één stoma te detecteren. De threshold hieronder is dezelfde als in de paper:

In [ ]:
threshold = 0.7

In [ ]:
stomata = []
offset = patch_size // 2
for x in np.arange(no_x_shifts + 1):
    for y in np.arange(no_y_shifts + 1):
        x_c = x * shift + offset
        y_c = y * shift + offset
        
        patch = image[x_c - offset:x_c + offset, y_c - offset:y_c + offset, :]
        patch = patch.astype("float32")
        patch /= 255
        
        y_model = model.predict(np.expand_dims(patch, axis=0))

        if y_model[0] > threshold:
            stomata.append([x_c, y_c])

### Clustering van de gedetecteerde stomata

Alle positief gelabelde patches worden geclusterd door middel van *mean shift clustering*. Deze techniek groepeert naburige (of zelfs overlappende) positief gelabelde patches waaruit de coördinaat van de effectieve stoma afgeleid wordt. Hiervoor kun je beroep doen op de module [MeanShift](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html) die beschikbaar is in [scikit-learn](https://scikit-learn.org).

In [ ]:
bandwidth = patch_size // 2

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(stomata)
stomata = np.array([[x[1], x[0]] for x in ms.cluster_centers_]) # x- en y-coördinaat van cluster_centers_ zijn verwisseld

### De resultaten plotten

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(image)
ax.plot(stomata[:,0], stomata[:,1], "xr", alpha=0.75, markeredgewidth=3, markersize=12)